In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required modules and import modules
!pip install transformers --quiet
!pip install sentencepiece --quiet
from transformers import AutoModel, AutoTokenizer
import string, re, random, json, itertools
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import unicodedata
from tqdm.auto import tqdm
import tensorflow as tf
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     |████████████████████████████████| 4.9 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
     |████████████████████████████████| 120 kB 72.2 MB/s 
     |████████████████████████████████| 1.3 MB 4.1 MB/s 


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
train_data_dir = '/content/drive/MyDrive/CS6910 Folder Team 29/Assignment 3/Machine Translation dataset/en-ta'
test_data_dir = '/content/drive/MyDrive/CS6910 Folder Team 29/Assignment 3/Machine Translation dataset/MT evaluation/en-ta'

In [4]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [5]:
with open(train_data_dir + "/train.ta", "r") as f:
    train_ta = f.read()
    train_ta = train_ta.lower()
    table = str.maketrans('', '', string.punctuation)
    train_ta = train_ta.translate(table)
    table = str.maketrans('', '', string.digits)
    train_ta = train_ta.translate(table)
    train_ta = train_ta.strip()
    #train_ta = unicodeToAscii(train_ta)
    train_ta = train_ta.split("\n")
    print(train_ta[:5])
    
with open(train_data_dir + "/train.en", "r") as f:
    train_en = f.read()
    train_en = train_en.lower()
    table = str.maketrans('', '', string.punctuation)
    train_en = train_en.translate(table)
    table = str.maketrans('', '', string.digits)
    train_en = train_en.translate(table)
    train_en = train_en.strip()
    #train_en = unicodeToAscii(train_en)
    train_en = train_en.split("\n")
    print(train_en[:5])

['என்றுதான் நான் சொல்ல வருகிறேன்', 'ஒவ்வொரு சுற்றுப்பயணமும் கடினமானது', 'பல வருடங்களாக அவர் அந்த நித்திய எரிநரக தண்டனையின் பயத்திலேயே வாழ்ந்து வந்தார்', 'அவர் நிதி அமைச்சர் அருண்ஜேட்லியின் முயற்சியை தொழில் உற்பத்தியில் ஈடுபட்டுள்ள மாநிலங்கள் மத்திய அரசின் உதவியைப் ஈடாக பெறுவதற்கு ஏற்ற கட்டளைவிதியை அமைத்துக்கொடுத்தற்கும் மற்றும் தொழில் உற்பத்தி இல்லாத மாநிலங்களுடன் தாங்கள் சமமாக வைக்கப்பட்டுள்ளார்கள் என்ற காழ்ப்பு உணர்ச்சிக்கு இடமில்லாமல் இருப்பதற்கேற்ற நிலையை ஏற்படுத்தியதற்காகவும் வெகுவாக பாராட்டினார்', 'சில கலை வரலாற்றாசிரியர்கள் அவர் ஒரு வருடத்திற்கு இரண்டு அல்லது மூன்று ஓவியங்களை மட்டுமே தயாரித்துள்ளதாக தெரிவித்திருக்கிறார்கள்']
['thats what i am saying', 'every tournament is difficult', 'one of the first questions flavio posed was does your religion teach that people are tormented in hellfire', 'he gave full credit to the union finance minister shri arun jaitley for working out an amicable formula wherein the manufacturing states would be compensated from the central share of funds

In [6]:
with open(test_data_dir + "/test.ta", "r") as f:
    test_ta = f.read()
    test_ta = test_ta.lower()
    table = str.maketrans('', '', string.punctuation)
    test_ta = test_ta.translate(table)
    table = str.maketrans('', '', string.digits)
    test_ta = test_ta.translate(table)
    test_ta = test_ta.strip()
    #test_ta = unicodeToAscii(test_ta)
    test_ta = test_ta.split("\n")
    print(test_ta[:5])
    
with open(test_data_dir + "/test.en", "r") as f:
    test_en = f.read()
    test_en = test_en.lower()
    table = str.maketrans('','',string.punctuation)
    test_en = test_en.translate(table)
    table = str.maketrans('','',string.digits)
    test_en = test_en.translate(table)
    test_en = test_en.strip()
    #test_en = unicodeToAscii(test_en)
    test_en = test_en.split("\n")
    print(test_en[:5])

['இந்த நிகழ்விற்கு சில மணி நேரம் முன்பு குர்திஷ் தேசியவாத தலைவரும் ஈராக்கிய ஜனாதிபதியுமான ஜலால் டாலிபானி ஈராக்கிற்கு கடைசி முறையாக வரும் அமெரிக்க ஜனாதிபதியை வரவேற்று நம் நாட்டை நமக்கு விடுவிக்க உதவிய மனிதர் இன்று நாம் ஜனநாயகம் மனித உரிமைகள் மற்றும் வளமையை படிப்படியாக நம் நாட்டில் பெறுவதற்கு உதவியவர் என்று பாராட்டினார்', 'மோசமடைந்துவரும் அந்நிய செலாவனி நெருக்கடியை எதிர்கொண்டுள்ள இலங்கை அரசாங்கம் நாட்டை பிணையெடுப்பதற்காக சர்வதேச நாணய நிதியத்திடம் இருந்து  பில்லியன் அமெரிக்க டொலர் கடனை எதிர்பார்க்கிறது', 'அங்குள்ள சன் டவர்ஸ் லாட்ஜில் அவருக்கு அறை ஏற்பாடாகியிருந்தது', ' வயது ஆஸ்திரேலியக் குடிமகனும் அமெரிக்காவில் வாழ்வபவருமான டக்லஸ் வூட் கடத்தப்பட்டு கொலை அச்சுறுத்தலுக்கு உட்பட்டிருப்பது அமெரிக்க ஆக்கிரமிப்பின் கீழ் ஈராக் எவ்வாறு உயிர்த்த நரகமாக உள்ளது என்பதற்கு மற்றொரு வெளிப்பாடாக இருக்கிறது', 'அப்பொழுது இல்க்கியாவும் ராஜா அனுப்பின மற்றவர்களும் அஸ்ராவின் குமாரனாகிய திக்வாதின் மகனான சல்லூம் என்னும் வஸ்திரசாலை விசாரிப்புக்காரன் மனைவியாகிய உல்தாள் என்னும் தீர்க்கதரிசியானவளிடத்திற்குப் போ

In [7]:
with open(test_data_dir + "/dev.ta", "r") as f:
    dev_ta = f.read()
    dev_ta = dev_ta.lower()
    table = str.maketrans('', '', string.punctuation)
    dev_ta = dev_ta.translate(table)
    table = str.maketrans('', '', string.digits)
    dev_ta = dev_ta.translate(table)
    dev_ta = dev_ta.strip()
    #dev_ta = unicodeToAscii(dev_ta)
    dev_ta = dev_ta.split("\n")
    print(dev_ta[:5])
    
with open(test_data_dir + "/dev.en", "r") as f:
    dev_en = f.read()
    dev_en = dev_en.lower()
    table = str.maketrans('', '', string.punctuation)
    dev_en = dev_en.translate(table)
    table = str.maketrans('', '', string.digits)
    dev_en = dev_en.translate(table)
    dev_en = dev_en.strip()
    #dev_en = unicodeToAscii(dev_en)
    dev_en = dev_en.split("\n")
    print(dev_en[:5])

['சவுலுக்கு இருந்த குமாரர் யோனத்தான் இஸ்வி மல்கிசூவா என்பவர்கள் அவனுடைய இரண்டு குமாரத்திகளில் மூத்தவள் பேர் மோப் இளையவள் பேர் மீகாள்', 'அஹமதினேஜாத் இஸ்ரேலை ஆதரிக்கும் போர் குற்றவாளிகள் பற்றியும் மற்றும் பிற நாடுகளில் தங்களுக்கே பாதுகாப்பு தேடிக் கொள்வதற்காக போர்களை நடத்திக் கொண்டிருப்பவர்கள் பற்றியும் மறைமுகமாக குறிப்பிட்டார்', 'சர்வதேச பொது மன்னிப்பு சபை ஒரு புலன் விசாரணை நடத்த வேண்டும் என்று கேட்டுக் கொண்டது', 'பாலஸ்தீன அரசு என்று கூறப்படும் ஒட்டுமொத்த நடவடிக்கைகளும் அது தொடர்பான அனைத்து தொடர் நடவடிக்கைகளும் எங்களது நிகழ்ச்சி நிரலில் இருந்து காலவரையின்றி விலக்கப்பட்டுவிட்டன', 'ஆயினும் இந்த அரசியல்வாதிகள் தேர்தல் நெருங்குவதன் காரணமாக எங்களை காண வருகின்றனர் என்று வெறுப்புணர்வுடன் அவர் கூறினார்']
['now the sons of saul were jonathan and ishui and melchishua and the names of his two daughters were these the name of the firstborn merab and the name of the younger michal', 'ahmadinejad obliquely referred to war criminals who back israel and those who fight wars in other countries 

In [8]:
num_words = 15
chosen_indices = [i for i in range(len(train_en)) if len(train_en[i].split()) < num_words and len(train_ta[i].split()) < num_words]

train_en = [train_en[i] for i in chosen_indices][:100000]
train_ta = [train_ta[i] for i in chosen_indices][:100000]

chosen_indices = [i for i in range(len(test_en)) if len(test_en[i].split()) < num_words and len(test_ta[i].split()) < num_words]
test_en = [test_en[i] for i in chosen_indices]
test_ta = [test_ta[i] for i in chosen_indices]

chosen_indices = [i for i in range(len(dev_en)) if len(dev_en[i].split()) < num_words and len(dev_ta[i].split()) < num_words]
dev_en = [dev_en[i] for i in chosen_indices]
dev_ta = [dev_ta[i] for i in chosen_indices]

print(len(train_en))
print(len(test_en))
print(len(dev_en))

100000
534
239


In [9]:
indic_tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert') 
indic_model = AutoModel.from_pretrained('ai4bharat/indic-bert')

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
indic_embedding = indic_model.get_input_embeddings()

In [11]:
vocab_ta = set([])

for sentence in tqdm(train_ta):
    tokens = indic_tokenizer.tokenize(sentence)
    for token in tokens:
        if token not in vocab_ta:
            vocab_ta.add(token)
            
for sentence in tqdm(test_ta):
    tokens = indic_tokenizer.tokenize(sentence)
    for token in tokens:
        if token not in vocab_ta:
            vocab_ta.add(token)
            
for sentence in tqdm(dev_ta):
    tokens = indic_tokenizer.tokenize(sentence)
    for token in tokens:
        if token not in vocab_ta:
            vocab_ta.add(token)

vocab_ta.add('[CLS]')
vocab_ta.add('[SEP]')
vocab_ta.add('[PAD]')

id_of_word_ta = {}
word_of_id_ta = {}
i=1
for word in vocab_ta:
    id_of_word_ta[word] = i
    word_of_id_ta[i] = word
    i = i+1

vocab_ta_len = len(vocab_ta) + 1

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/534 [00:00<?, ?it/s]

  0%|          | 0/239 [00:00<?, ?it/s]

In [12]:
#vocab_ta_len = 7000

In [13]:
print(vocab_ta_len)

2480


In [14]:
with open('id_of_word_ta.json', 'w') as fp:
    json.dump(id_of_word_ta, fp)
    
with open('word_of_id_ta.json', 'w') as fp:
    json.dump(word_of_id_ta, fp)

In [15]:
"""
with open('../input/preprocessed-final/id_of_word_ta (3).json', 'r') as fp:
    id_of_word_ta = json.load(fp)
    
with open('../input/preprocessed-final/word_of_id_ta (3).json', 'r') as fp:
    word_of_id_ta = json.load(fp)
"""

"\nwith open('../input/preprocessed-final/id_of_word_te (3).json', 'r') as fp:\n    id_of_word_te = json.load(fp)\n    \nwith open('../input/preprocessed-final/word_of_id_te (3).json', 'r') as fp:\n    word_of_id_te = json.load(fp)\n"

In [16]:
number_train = len(train_en)
print(number_train)

100000


In [17]:
vocab_en = set([])

for sentence in tqdm(train_en):
    words = sentence.split()
    for word in words:
        if word not in vocab_en:
            vocab_en.add(word)
            
for sentence in tqdm(test_en):
    words = sentence.split()
    for word in words:
        if word not in vocab_en:
            vocab_en.add(word)
            
for sentence in tqdm(dev_en):
    words = sentence.split()
    for word in words:
        if word not in vocab_en:
            vocab_en.add(word)

vocab_en.add('[PAD]')

id_of_word_en = {}
word_of_id_en = {}
i=1
for word in vocab_en:
    id_of_word_en[word] = i
    word_of_id_en[i] = word
    i = i+1

vocab_en_len = len(vocab_en) + 1

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/534 [00:00<?, ?it/s]

  0%|          | 0/239 [00:00<?, ?it/s]

In [18]:
#vocab_en_len = 117709

In [19]:
print(vocab_en_len)

30931


In [20]:
with open('id_of_word_en.json', 'w') as fp:
    json.dump(id_of_word_en, fp)
    
with open('word_of_id_en.json', 'w') as fp:
    json.dump(word_of_id_en, fp)

In [21]:
"""
with open('../input/preprocessed-final/id_of_word_en (3).json', 'r') as fp:
    id_of_word_en = json.load(fp)
    
with open('../input/preprocessed-final/word_of_id_en (3).json', 'r') as fp:
    word_of_id_en = json.load(fp)
"""

"\nwith open('../input/preprocessed-final/id_of_word_en (3).json', 'r') as fp:\n    id_of_word_en = json.load(fp)\n    \nwith open('../input/preprocessed-final/word_of_id_en (3).json', 'r') as fp:\n    word_of_id_en = json.load(fp)\n"

In [22]:
embedding_matrix_ta = np.zeros((vocab_ta_len, 128))
for word, i in tqdm(id_of_word_ta.items()):
    token_id = indic_tokenizer.convert_tokens_to_ids(word)
    embedding_matrix_ta[i] = indic_embedding(torch.tensor([token_id])).detach()
embedding_matrix_ta = torch.FloatTensor(embedding_matrix_ta)

  0%|          | 0/2479 [00:00<?, ?it/s]

In [23]:
src_len = 15 # length of source
tgt_len = 101 # length of target

embedding_dim = 128
d_ff = 2048  # FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_layers = 4  # number of Encoder of Decoder Layer
n_heads = 4  # number of heads in Multi-Head Attention

In [24]:
def make_batch(sentences):
    input_batch = [[id_of_word_en[n] for n in sentences[0].split()]]
    output_batch = [[id_of_word_ta[n] for n in indic_tokenizer.tokenize(sentences[1])]]
    target_batch = [[id_of_word_ta[n] for n in indic_tokenizer.tokenize(sentences[2])]]
    return torch.LongTensor(input_batch), torch.LongTensor(output_batch), torch.LongTensor(target_batch)

def get_sinusoid_encoding_table(n_position, embedding_dim):
    def cal_angle(position, hid_idx):
        return position / np.power(10000, 2 * (hid_idx // 2) / embedding_dim)
    def get_posi_angle_vec(position):
        return [cal_angle(position, hid_j) for hid_j in range(embedding_dim)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(n_position)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1
    return torch.FloatTensor(sinusoid_table)

def get_attn_pad_mask(seq_q, seq_k):
    # print(seq_q)
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

def get_attn_subsequent_mask(seq):
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequent_mask = np.triu(np.ones(attn_shape), k=1)
    subsequent_mask = torch.from_numpy(subsequent_mask).byte()
    return subsequent_mask

In [25]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(embedding_dim, d_k * n_heads)
        self.W_K = nn.Linear(embedding_dim, d_k * n_heads)
        self.W_V = nn.Linear(embedding_dim, d_v * n_heads)
        self.linear = nn.Linear(n_heads * d_v, embedding_dim)
        self.layer_norm = nn.LayerNorm(embedding_dim)

    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x embedding_dim], k: [batch_size x len_k x embedding_dim], v: [batch_size x len_k x embedding_dim]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = self.linear(context)
        return self.layer_norm(output + residual), attn # output: [batch_size x len_q x embedding_dim]

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=embedding_dim, kernel_size=1)
        self.layer_norm = nn.LayerNorm(embedding_dim)

    def forward(self, inputs):
        residual = inputs # inputs : [batch_size, len_q, embedding_dim]
        output = nn.ReLU()(self.conv1(inputs.transpose(1, 2)))
        output = self.conv2(output).transpose(1, 2)
        return self.layer_norm(output + residual)

class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x embedding_dim]
        return enc_outputs, attn

class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.dec_enc_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs)
        return dec_outputs, dec_self_attn, dec_enc_attn

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(vocab_en_len, embedding_dim)
        self.pos_emb = nn.Embedding.from_pretrained(get_sinusoid_encoding_table(src_len+1, embedding_dim),freeze=True)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs, enc_batch): # enc_inputs : [batch_size x source_len]
        enc_outputs = self.src_emb(enc_inputs) 
        enc_outputs += self.pos_emb(enc_batch)
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
        enc_self_attns = []
        for layer in self.layers:
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding.from_pretrained(embedding_matrix_ta,freeze=True)
        self.pos_emb = nn.Embedding.from_pretrained(get_sinusoid_encoding_table(tgt_len+1, embedding_dim),freeze=True)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs, dec_batch): # dec_inputs : [batch_size x target_len]
        dec_outputs = self.tgt_emb(dec_inputs)
        dec_outputs += self.pos_emb(dec_batch)
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs)
        dec_self_attn_subsequent_mask = get_attn_subsequent_mask(dec_inputs)
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequent_mask), 0)

        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        return dec_outputs, dec_self_attns, dec_enc_attns

class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.projection = nn.Linear(embedding_dim, vocab_ta_len, bias=False)
    def forward(self, enc_inputs, dec_inputs, enc_batch, dec_batch):
        enc_outputs, enc_self_attns = self.encoder(enc_inputs, enc_batch)
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs, dec_batch)
        dec_logits = self.projection(dec_outputs) # dec_logits : [batch_size x vocab_en_len x vocab_ta_len]
        return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns

In [26]:
def greedy_decoder(model, enc_input, start_symbol):
    enc_outputs, enc_self_attns = model.encoder(enc_input)
    dec_input = torch.zeros(1, 5).type_as(enc_input.data)
    next_symbol = start_symbol
    for i in range(0, 5):
        dec_input[0][i] = next_symbol
        dec_outputs, _, _ = model.decoder(dec_input, enc_input, enc_outputs)
        projected = model.projection(dec_outputs)
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]
        next_word = prob.data[i]
        next_symbol = next_word.item()
    return dec_input

def showgraph(attn):
    attn = attn[-1].squeeze(0)[0]
    attn = attn.squeeze(0).data.numpy()
    fig = plt.figure(figsize=(n_heads, n_heads)) # [n_heads, n_heads]
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attn, cmap='viridis')
    ax.set_xticklabels(['']+sentences[0].split(), fontdict={'fontsize': 14}, rotation=90)
    ax.set_yticklabels(['']+sentences[2].split(), fontdict={'fontsize': 14})
    plt.show()

In [27]:
def collate_func(batch):
    src_batch, out_batch, tgt_batch = [], [], []
    enc_batch, dec_batch = [], []
    for sample in batch:
        src_sample = sample['input']
        out_sample = sample['output']
        tgt_sample = sample['target']
        enc_sample = sample['enc_pos']
        dec_sample = sample['dec_pos']
    
        src_batch.append(src_sample)
        out_batch.append(out_sample)
        tgt_batch.append(tgt_sample)
        enc_batch.append(enc_sample)
        dec_batch.append(dec_sample)
    
    src_batch = list(zip(*itertools.zip_longest(*src_batch, fillvalue=0)))
    src_batch = [list(x) for x in src_batch]
    out_batch = list(zip(*itertools.zip_longest(*out_batch, fillvalue=0)))
    out_batch = [list(x) for x in out_batch]
    tgt_batch = list(zip(*itertools.zip_longest(*tgt_batch, fillvalue=0)))
    tgt_batch = [list(x) for x in tgt_batch]
    enc_batch = list(zip(*itertools.zip_longest(*enc_batch, fillvalue=0)))
    enc_batch = [list(x) for x in enc_batch]
    dec_batch = list(zip(*itertools.zip_longest(*dec_batch, fillvalue=0)))
    dec_batch = [list(x) for x in dec_batch]
    
    return torch.LongTensor(src_batch), torch.LongTensor(out_batch), torch.LongTensor(tgt_batch), torch.LongTensor(enc_batch), torch.LongTensor(dec_batch)

In [28]:
class textDataset(torch.utils.data.Dataset):

    def __init__(self, list_en, list_ta):
        self.list_en = list_en
        self.list_ta = list_ta

    def __len__(self):
        return len(self.list_en)

    def __getitem__(self, idx):
        sentence = self.list_en[idx]
        input_tansor = [id_of_word_en[word] for word in sentence.split() if word in id_of_word_en]
        enc_pos = list(range(1, len(input_tansor)+1))
        sentence = self.list_ta[idx]
        tokens = indic_tokenizer.tokenize(sentence)
        tokens2 = tokens[:]
        tokens2.insert(0, '[CLS]')
        tokens.append('[SEP]')
        token_ids = [id_of_word_ta[word] for word in tokens if word in id_of_word_ta]
        token_ids2 = [id_of_word_ta[word] for word in tokens2 if word in id_of_word_ta]
        dec_pos = list(range(1, len(token_ids2)))
        dec_pos.insert(0, len(token_ids2))
        sample = {'input': input_tansor, 'output': token_ids2, 'target': token_ids, 'enc_pos' : enc_pos, 'dec_pos' : dec_pos}
        return sample

In [29]:
batch_size = 128

In [30]:
train_dataset = textDataset(train_en, train_ta)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, collate_fn = collate_func)
dev_dataset = textDataset(dev_en, dev_ta)
dev_data_loader = torch.utils.data.DataLoader(dev_dataset, batch_size = batch_size, collate_fn = collate_func)

In [31]:
model = Transformer()

criterion = nn.CrossEntropyLoss(ignore_index = 0)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

lowest_val_loss = float('inf')
for epoch in range(7):
    for enc_inputs, dec_inputs, target_batch, enc_batch, dec_batch in tqdm(train_data_loader):
        optimizer.zero_grad()
        outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs, enc_batch, dec_batch)
        loss = criterion(outputs, target_batch.contiguous().view(-1))
        # print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
        loss.backward()
        optimizer.step()
    torch.save(model.state_dict(), './saved_model.pt')
        

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

In [32]:
model.load_state_dict(torch.load("./saved_model.pt"))

<All keys matched successfully>

In [33]:
def greedy_decoder(model, enc_input, enc_batch, start_symbol):
    enc_outputs, enc_self_attns = model.encoder(enc_input, enc_batch)
    dec_input = torch.zeros(1, 100).type_as(enc_input.data)
    dec_batch = torch.zeros(1, 100).type_as(enc_input.data)
    next_symbol = start_symbol
    pos_ctr = list(range(1, 100))
    pos_ctr.insert(0, 100)
    l = []
    for i in range(100):
        dec_input[0][i] = next_symbol
        dec_batch[0][i] = pos_ctr[i]
        dec_outputs, _, _ = model.decoder(dec_input, enc_input, enc_outputs, dec_batch)
        projected = model.projection(dec_outputs)
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]
        next_word = prob.data[i]
        next_symbol = next_word.item()
        l.append(next_symbol)
        if next_symbol == id_of_word_ta['[SEP]']:
            break
            
    return dec_input, dec_batch

In [34]:
test_dataset = textDataset(test_en, test_ta)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, collate_fn = collate_func)

In [35]:
l = []
for enc_inputs, dec_inputs, target_batch, enc_batch, dec_batch in tqdm(test_data_loader):
    greedy_dec_input, greedy_dec_batch = greedy_decoder(model, enc_inputs, enc_batch, start_symbol=id_of_word_ta["[CLS]"])
    predict, _, _, _ = model(enc_inputs, greedy_dec_input, enc_batch, greedy_dec_batch)
    predict = predict.data.max(1, keepdim=True)[1]
    temp = []
    for n in predict.squeeze():
        if word_of_id_ta[n.item()] == '[SEP]':
            break
        temp.append(word_of_id_ta[n.item()])
    l.append(temp)

  0%|          | 0/534 [00:00<?, ?it/s]

In [36]:
actual = [[indic_tokenizer.tokenize(sentence)] for sentence in test_ta]

smoothie = SmoothingFunction().method7

bleu_1 = corpus_bleu(actual, l, weights=(1.0, 0, 0, 0))
bleu_2 = corpus_bleu(actual, l, weights=(0.5, 0.5, 0, 0))
bleu_3 = corpus_bleu(actual, l, weights=(0.33, 0.33, 0.33, 0))
bleu_4 = corpus_bleu(actual, l, weights=(0.25, 0.25, 0.25, 0.25))

print("Test Metrics:")
print("BLEU-1: %f" % bleu_1)
print("BLEU-2: %f" % bleu_2)
print("BLEU-3: %f" % bleu_3)
print("BLEU-4: %f\n\n" % bleu_4)

sbleu_1 = corpus_bleu(actual, l, weights=(1.0, 0, 0, 0), smoothing_function = smoothie)
sbleu_2 = corpus_bleu(actual, l, weights=(0.5, 0.5, 0, 0), smoothing_function = smoothie)
sbleu_3 = corpus_bleu(actual, l, weights=(0.33, 0.33, 0.33, 0), smoothing_function = smoothie)
sbleu_4 = corpus_bleu(actual, l, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function = smoothie)

print("Smoothened Test Metrics:")
print("Smooth BLEU-1: %f" % sbleu_1)
print("Smooth BLEU-2: %f" % sbleu_2)
print("Smooth BLEU-3: %f" % sbleu_3)
print("Smooth BLEU-4: %f" % sbleu_4)

Test Metrics:
BLEU-1: 0.6314
BLEU-2: 0.6031
BLEU-3: 0.598
BLEU-4: 0.5491


Smoothened Test Metrics:
Smooth BLEU-1: 0.6476
Smooth BLEU-2: 0.6151
Smooth BLEU-3: 0.6002
Smooth BLEU-4: 0.5562
